In [10]:
import json
import pandas as pd
from collections import defaultdict
import re, os

In [4]:
def read_json(file_path):
   with open(file_path, 'r') as json_file:
    data = json.load(json_file)
    return data 

In [5]:
version = "3"
method = "PO"

root_path = "D:\OneDrive - Delft University of Technology\Thesis\DEBBIE_STARFISH_2223"
ff_file_path = "fformations_each_version/" + method + "_f-formations_psecond_" + version + ".json"
ge_file_path = "fformations_each_version/" + method + "_Group_encoding_" + version + ".json"
gc_file_path = "fformations_each_version/" + method + "_aggregated_frequent_formations_" + version + ".json"


if version == "3":
    directory_path = root_path + "\Version 3_ Kalman_Fixed_Data + Overlap Filter\Synched_Data_GR0_2to2_ANGLE45\COTALK"
    file_pattern = re.compile(r'DAYCOTALK_(\d{6})_COTALK0_22_DEN_072924_V21588968411.CSV') #3
    
elif version == "2":
    directory_path = root_path + "\Version 2_Kalman_Fixed_Data\Synched_Data_GR0_2to2_ANGLE45\COTALK"
    file_pattern = re.compile(r'DAYCOTALK_(\d{6})_COTALK0_22_DEN_072224_V22072780461.CSV') # 2
    
# elif version == "1":
#     ge_file_path = 'fformations_each_version/P_Group_encoding_1.json'
    
else:
    print("FILE NOT FOUND")
    
files = [f for f in os.listdir(directory_path) if file_pattern.match(f)]
files, len(files)

(['DAYCOTALK_013023_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_020123_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_031323_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_031523_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_041723_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_041923_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_061523_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_101922_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_102122_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_111422_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_111622_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_120522_COTALK0_22_DEN_072924_V21588968411.CSV',
  'DAYCOTALK_120722_COTALK0_22_DEN_072924_V21588968411.CSV'],
 13)

In [6]:
fformations_ps = read_json(ff_file_path)
group_encodings = read_json(ge_file_path)
group_count = read_json(gc_file_path)

In [31]:
fformations_ps

{'1': {'2023-01-30 09:47:00.100000': {'0': ['32', '29'],
   '1': ['46', '28'],
   '2': ['41', '33']},
  '2023-01-30 09:47:01.100000': {'0': ['32', '33', '29'], '1': ['46', '28']},
  '2023-01-30 09:47:02.100000': {'0': ['32', '33', '29'], '1': ['46', '28']},
  '2023-01-30 09:47:03.100000': {'0': ['41', '33'], '1': ['31', '46']},
  '2023-01-30 09:47:04.100000': {'0': ['41', '33'], '1': ['31', '46']},
  '2023-01-30 09:47:05.100000': {'0': ['32', '33', '29'], '1': ['46', '28']},
  '2023-01-30 09:47:06.100000': {'0': ['32', '33'], '1': ['27', '28']},
  '2023-01-30 09:47:07.100000': {'0': ['31', '46'], '1': ['33', '29']},
  '2023-01-30 09:47:08.100000': {'0': ['33', '29'], '1': ['31', '46']},
  '2023-01-30 09:47:09.100000': {'0': ['32', '33', '29'], '1': ['46', '28']},
  '2023-01-30 09:47:10.100000': {'0': ['45', '32'], '1': ['46', '28']},
  '2023-01-30 09:47:11.100000': {'0': ['44', '33', '29'],
   '1': ['46', '28'],
   '2': ['45', '32']},
  '2023-01-30 09:47:12.100000': {'0': ['44', '33', 

In [6]:
# The correct one
map_HL = {    
    '27': 'HL',
    '28': 'HL',
    '29': 'HL',
    '30': 'HL',
    '31': 'HL',
    '32': 'TH',
    '33': 'HL',
    '41': 'TH',
    '42': 'TH',
    '43': 'TH',
    '44': 'TH',
    '45': 'TH',
    '46': 'TH'
}

In [ ]:
HL_ratio = {}
homo_degree = {}
group_size = {}

for group_list, encoder in group_encodings.items():
    HL_count = 0
    TH_count = 0
    group_list = json.loads(group_list)
    for person in list(group_list):
        if map_HL.get(person) == "HL":
            HL_count += 1
        else:
            TH_count += 1
    HL_ratio[encoder] = HL_count/len(group_list)
    homo_degree[encoder] = max(HL_count,TH_count)/len(group_list)
    group_size[encoder] = len(group_list)
HL_ratio, homo_degree

In [ ]:
person_group_count = defaultdict(lambda: defaultdict(int))

for day, all_fformations in fformations_ps.items():
    for timestamp, fformations in all_fformations.items():
        for group_id, group_list in fformations.items():
            sorted_group_list = sorted(group_list, key=lambda x: int(x))
            group_list_str = json.dumps(sorted_group_list)
            # to create double quote
            group_identifier = group_encodings.get(group_list_str)
            for person in sorted_group_list:
                person_group_count[person][group_identifier] += 1

for person, groups in person_group_count.items():
    for group_id, count in groups.items():
        print(f"Person {person} appeared in group {group_id} {count} times.")              
            

In [ ]:
# count the total time each person is in the classroom for all days

# File pattern to match
files = [f for f in os.listdir(directory_path) if file_pattern.match(f)]

day = 1
result_dict = []
for file in files:
    # read file
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)
    print(file)
    
    df['SUBJECTID'] = df['SUBJECTID'].str.extract(r'(\d+)$')
    df['TIME'] = pd.to_datetime(df['TIME'])
    start_time_df = df.groupby(['SUBJECTID'])['TIME'].min().reset_index()
    end_time_df = df.groupby(['SUBJECTID'])['TIME'].max().reset_index()
    
    # Merge start and end times
    time_range_df = pd.merge(start_time_df, end_time_df, on=['SUBJECTID'])
    time_range_df.columns = ['SUBJECTID', 'Start_Time', 'End_Time']

    # Calculate total time
    time_range_df['Total_Time'] = time_range_df['End_Time'] - time_range_df['Start_Time']

    # # Apply the function to calculate bad time
    # time_range_df['Bad_Time'] = time_range_df.apply(calculate_bad_time, axis=1)

    # # Adjust total time by subtracting bad time
    # time_range_df['Adjusted_Total_Time'] = time_range_df['Total_Time']  - time_range_df['Bad_Time']
    time_range_df['Adjusted_Total_Time'] = time_range_df['Total_Time'].dt.total_seconds()
    result_dict.append(time_range_df)
final_results_df = pd.concat(result_dict, ignore_index=True)
final_results_df

In [ ]:
sum_times_df = final_results_df.groupby('SUBJECTID')['Adjusted_Total_Time'].sum().reset_index()

# Rename the columns for clarity
sum_times_df.columns = ['SUBJECTID', 'Total_Adjusted_Time']

# Print the result
print(sum_times_df)

In [ ]:
# Convert sum_times_df to a dictionary for easier lookup
total_time_dict = sum_times_df.set_index('SUBJECTID')['Total_Adjusted_Time'].to_dict()

# Initialize list to hold rows for the new DataFrame
rows = []

# Process each person in person_group_count
for person, group_counts in person_group_count.items():
    if person in total_time_dict:
        total_time = total_time_dict[person]
        for group, count in group_counts.items():
            # Add row to the list
            rows.append({
                'person': person,
                'group': group,
                'count': count,
                'Total_Adjusted_Time': total_time,
                'diagnosisPerson': map_HL.get(person, 'Unknown'),
                'HL_ratio': HL_ratio.get(group, 'Unknown'),  # Use 'Unknown' if group is not found
                'Homophily_degree': homo_degree.get(group, 'Unknown'),  # Use 'Unknown' if group is not found
                'Group_Size': group_size.get(group, 'Unknown'),  # Use 'Unknown' if group is not found
                'Social_contact_ratio': count / total_time  # Calculate ratio
            })

# Create the new DataFrame
new_df = pd.DataFrame(rows)

save_file =  method + "_individual_group_sclag_" + version + ".csv"
# Print the result
new_df.to_csv(save_file, index=False)
new_df.head()